In [25]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from  sklearn.metrics import roc_auc_score
import gc
import lightgbm as lgb
import sklearn
from catboost import CatBoostClassifier, Pool
import catboost

Очистка кода

In [26]:
train = pd.read_csv('train_dataset_Самолет.csv', low_memory=False)
test = pd.read_csv('test.csv', low_memory=False, sep = ';')
train = pd.concat([train, test]).reset_index(drop = True)

In [27]:
duplicates_cols = train.T.duplicated()
drop_cols_0 = duplicates_cols[duplicates_cols].index
train = train.drop(columns = drop_cols_0)

In [28]:
# cols = train.columns[3:]
# drop_cols = []
# for i in tqdm(np.arange(len(cols))):
#     all_cols = cols[max(0, i - 30): i].tolist() + cols[max(0, i + 1): i + 30].tolist()
#     all_cols = sorted(set(all_cols) - set(drop_cols))
#     tmp_df = train[all_cols].fillna(-999.4).values
#     check = ((train[cols[i]].fillna(-999.4).values[:, None] != tmp_df) * (~train[cols[i]].isnull()).values[:, None]).sum(0)
#     if check.min() == 0:
#         drop_cols += [cols[i]]

drop_cols_1 = ['col1', 'col5', 'col7', 'col25', 'col27', 'col29', 'col31', 'col33', 'col35', 'col45', 'col46', 'col47', 'col49', 'col50', 'col51', 'col54', 'col57', 'col58', 'col59', 'col62', 'col85', 'col86', 'col87', 'col89', 'col93', 'col94', 'col95', 'col101', 'col105', 'col106', 'col107', 'col109', 'col129', 'col130', 'col131', 'col137', 'col138', 'col139', 'col141', 'col142', 'col145', 'col146', 'col147', 'col153', 'col154', 'col155', 'col157', 'col158', 'col161', 'col162', 'col163', 'col169', 'col170', 'col171', 'col173', 'col181', 'col182', 'col183', 'col189', 'col190', 'col191', 'col201', 'col202', 'col203', 'col205', 'col209', 'col210', 'col211', 'col221', 'col222', 'col223', 'col233', 'col234', 'col237', 'col238', 'col239', 'col241', 'col457', 'col458', 'col459', 'col465', 'col466', 'col467', 'col513', 'col514', 'col515', 'col518', 'col521', 'col522', 'col523', 'col529', 'col530', 'col531', 'col537', 'col538', 'col539', 'col545', 'col546', 'col547', 'col553', 'col554', 'col555', 'col561', 'col562', 'col563', 'col569', 'col570', 'col571', 'col577', 'col578', 'col579', 'col585', 'col586', 'col587', 'col593', 'col594', 'col595', 'col601', 'col602', 'col603', 'col609', 'col610', 'col611', 'col773', 'col793', 'col794', 'col795', 'col801', 'col802', 'col803', 'col809', 'col810', 'col811', 'col817', 'col819', 'col905', 'col906', 'col966', 'col1005', 'col1017', 'col1029', 'col1030', 'col1031', 'col1032', 'col1033', 'col1034', 'col1035', 'col1037', 'col1038', 'col1039', 'col1041', 'col1045', 'col1046', 'col1047', 'col1053', 'col1054', 'col1055', 'col1061', 'col1062', 'col1063', 'col1070', 'col1071', 'col1073', 'col1077', 'col1081', 'col1082', 'col1089', 'col1090', 'col1094', 'col1117', 'col1118', 'col1119', 'col1121', 'col1122', 'col1123', 'col1125', 'col1126', 'col1133', 'col1134', 'col1137', 'col1145', 'col1165', 'col1166', 'col1167', 'col1174', 'col1175', 'col1177', 'col1179', 'col1181', 'col1197', 'col1230', 'col1234', 'col1237', 'col1250', 'col1273', 'col1274', 'col1277', 'col1278', 'col1281', 'col1285', 'col1317', 'col1321', 'col1327', 'col1341', 'col1347', 'col1357', 'col1359', 'col1373', 'col1377', 'col1402', 'col1425', 'col1437', 'col1445', 'col1446', 'col1447', 'col1449', 'col1450', 'col1451', 'col1647', 'col1649', 'col1650', 'col1651', 'col1661', 'col1664', 'col1667', 'col1682', 'col1685', 'col1688', 'col1691', 'col1694', 'col1706', 'col1707', 'col1708', 'col1709', 'col1710', 'col1711', 'col1712', 'col1713', 'col1714', 'col1715', 'col1716', 'col1717', 'col1718', 'col1719', 'col1720', 'col1730', 'col1733', 'col1736', 'col1739', 'col1742', 'col1743', 'col1744', 'col1754', 'col1757', 'col1760', 'col1763', 'col1766', 'col1778', 'col1779', 'col1780', 'col1781', 'col1782', 'col1783', 'col1784', 'col1785', 'col1786', 'col1787', 'col1788', 'col1789', 'col1790', 'col1791', 'col1792', 'col1802', 'col1803', 'col1804', 'col1805', 'col1806', 'col1807', 'col1808', 'col1809', 'col1810', 'col1811', 'col1812', 'col1813', 'col1814', 'col1815', 'col1816', 'col2047', 'col2065', 'col2066', 'col2067', 'col2137', 'col2196', 'col2197', 'col2242', 'col2463', 'col2464', 'col2465', 'col2466', 'col2467', 'col2468', 'col2469', 'col2471', 'col2472', 'col2473', 'col2474', 'col2475', 'col2477', 'col2543', 'col2562', 'col2585', 'col2586', 'col2587', 'col2591', 'col2594', 'col2595', 'col2596', 'col2597', 'col2598', 'col2599', 'col2600', 'col2606', 'col2609', 'col2610', 'col2611', 'col2615', 'col2618', 'col2619', 'col2620', 'col2621', 'col2622', 'col2623', 'col2624', 'col2630', 'col2631', 'col2632', 'col2633', 'col2634', 'col2635', 'col2639', 'col2658']
train = train.drop(columns = drop_cols_1)

In [29]:
isnull_samples = (~train.isnull()).sum(0)
drop_cols_2 = isnull_samples[isnull_samples < train.shape[0] * 0.02].index
train = train.drop(columns = drop_cols_2)

In [30]:
types = train.dtypes.astype('str')
for col in types[types == 'object'].index[1:]:
    train['cat_' + col] = train[col].astype('category').cat.codes.tolist()

C:\Users\arefi\AppData\Local\Temp\ipykernel_5436\3192305251.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train['cat_' + col] = train[col].astype('category').cat.codes.tolist()
C:\Users\arefi\AppData\Local\Temp\ipykernel_5436\3192305251.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train['cat_' + col] = train[col].astype('category').cat.codes.tolist()
C:\Users\arefi\AppData\Local\Temp\ipykernel_5436\3192305251.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inse

Валидация

In [31]:
gkf = sklearn.model_selection.GroupKFold(3)
split_list = []
for date in ['2023-02-01', '2023-01-01']:
    val_client = train[train.report_date >= date ]['client_id'].values
    for _, tmp_index in gkf.split(val_client, val_client, val_client):
        fold_val_client = val_client[tmp_index]
        train_index =  train[(train.report_date >= '2021-07-01' ) & (train.report_date < date ) & (~train.client_id.isin(fold_val_client))].index
        val_index =  train[(train.report_date <= '2023-02-01' ) & (train.report_date >= date ) & (train.client_id.isin(fold_val_client))].index
        split_list += [(train_index, val_index)]
        print(len(train_index), len(val_index))

10133 361
10329 378
10348 398
9077 783
9036 768
9006 745


Обучение

In [32]:
def lgb_train(data, target, ltr, split_list, param, v_e = 0, n_e = 50, weight_col = []):
    pred = pd.DataFrame()
    pred_val = np.zeros(ltr)
    fi = np.zeros(data.shape[1])
    score = []
    bst_list = []
    pred_train = pd.DataFrame()
    j = 0
    full_tr = lgb.Dataset(np.array(data), np.array(target))

    cv = lgb.cv(param, full_tr, 1000, folds = split_list,eval_train_metric = True,
                callbacks = [lgb.early_stopping(400, verbose = None), 
                                                                      lgb.log_evaluation(50)] )
    trees = np.argmax(cv['valid auc-mean']) + 1
    print(trees, np.max(cv['valid auc-mean']))

    for i , (train_index, test_index) in enumerate(split_list):

        tr = lgb.Dataset(data.loc[train_index], target[train_index])
        te = lgb.Dataset(data.loc[test_index], target[test_index], reference=tr)

        bst = lgb.train(param, tr, num_boost_round=trees)

        pred[str(i)] = bst.predict(np.array(data)[ltr:, :])
        pred_val[test_index] = bst.predict(np.array(data)[test_index]) 

        score += [roc_auc_score(np.array(target)[test_index], pred_val[test_index])]
        bst_list += [bst]
        fi += np.array(bst.feature_importance(importance_type = 'gain'))
        print(i+1, np.mean(score))
        gc.collect()
        del tr, te
    pred_train[str(j)] = pred_val
    return bst_list, pred_train, pred, score, fi

In [33]:
def occur(x):
    x1 = pd.Series(np.arange(len(x)))
    x1.index = x.index
    x.update(np.arange(len(x)))
    return x1

train['time_occur'] = train.sort_values('report_date').groupby('client_id')['target'].apply(lambda x: occur(x))
train['date_diff'] = pd.to_datetime(train['report_date']).dt.month + (pd.to_datetime(train['report_date']).dt.year - 2021) * 12
train['month'] = pd.to_datetime(train['report_date']).dt.month

In [34]:
param_lgb = {
'learning_rate': 0.1,
'metric':'auc',
'num_leaves': 16,
'lambda_l1' : 10,
'min_data_in_leaf' : 50,
'max_bin':200,
'extra_trees' : True,
'verbosity' : -1, 
'objective': 'binary'
}

In [35]:
bst_list_lgb = []
for seed in [0, 1, 2, 3, 4]:
    train_cols = ['col2663', 'time_occur', 'col2359', 'col1456', 'col1836', 'col2212', 'col1455', 'col2215', 'col2311', 'col2470', 'col1942', 'col2654', 'col2400', 'col2316', 'col1466', 'col1459', 'col1826', 'col1948'] 
    param_lgb['seed'] = seed
    bst_list, pred_train, pred_test, score, fi = lgb_train(train[train_cols], train['target'], len(train), split_list, param_lgb, 10,)
    bst_list_lgb += [bst_list]

[50]	cv_agg's train auc: 0.851252 + 0.00136169	cv_agg's valid auc: 0.927724 + 0.0365249
[100]	cv_agg's train auc: 0.86202 + 0.000821706	cv_agg's valid auc: 0.933901 + 0.0335551
[150]	cv_agg's train auc: 0.867097 + 0.000776109	cv_agg's valid auc: 0.933366 + 0.037549
[200]	cv_agg's train auc: 0.868628 + 0.000399804	cv_agg's valid auc: 0.932406 + 0.0378805
[250]	cv_agg's train auc: 0.869807 + 0.000356673	cv_agg's valid auc: 0.932174 + 0.0379359
[300]	cv_agg's train auc: 0.870633 + 0.000351653	cv_agg's valid auc: 0.932187 + 0.0376106
[350]	cv_agg's train auc: 0.871169 + 0.000498134	cv_agg's valid auc: 0.931737 + 0.0385046
[400]	cv_agg's train auc: 0.871517 + 0.000642036	cv_agg's valid auc: 0.931267 + 0.0389312
[450]	cv_agg's train auc: 0.871831 + 0.00066697	cv_agg's valid auc: 0.931329 + 0.0388786
[500]	cv_agg's train auc: 0.872032 + 0.000755737	cv_agg's valid auc: 0.931275 + 0.0389167
109 0.9354407024286729
1 0.9663865546218489
2 0.9676527367703839
3 0.9555719099836747
4 0.952817774447516

In [36]:
tmp_df = train[train.report_date >= '2023-03-01'][train_cols]
predict_lgb_test = []
for bst_list in bst_list_lgb:
    for bst in bst_list[3:]:
        predict_lgb_test += [bst.predict(tmp_df)]
lgb_predict = np.mean(predict_lgb_test, 0)

In [37]:
def cat_train(data, target, ltr, split_list, param, v_e = 0, n_e = 50, trees = 700):
    pred = pd.DataFrame()
    pred_val = np.zeros(ltr)
    fi = np.zeros(data.shape[1])
    score = []
    bst_list = []
    pred_train = pd.DataFrame()
    j = 0

    tr = Pool(data.values, label= target.values)
    cv = catboost.cv(tr, params_cat, iterations = 250,logging_level='Silent', folds = split_list)
    trees = cv[cv['test-AUC-mean'] == cv['test-AUC-mean'].max()]['iterations'].iloc[0]
    print(trees, cv['test-AUC-mean'].max())
    # trees = 2
    for i , (train_index, test_index) in enumerate(split_list):
        tr = Pool(data.loc[train_index].values, label= target.values[train_index])
        bst = catboost.train(tr, param, num_boost_round=trees , verbose = 0)

        pred[str(i)] = bst.predict(np.array(data.loc[ltr:]), prediction_type = 'Probability')[:, 1]
        pred_val[test_index] = bst.predict(np.array(data.loc[test_index]), prediction_type = 'Probability')[:, 1] 

        score += [roc_auc_score(np.array(target)[test_index], pred_val[test_index])]
        bst_list += [bst]
        fi += np.array(bst.get_feature_importance())
        print(i+1, np.mean(score))
#         print()
        gc.collect()
        del tr
        # , te
    pred_train[str(j)] = pred_val
    return bst_list, pred_train, pred, score, fi

In [38]:
best_cols = ['col2663', 'time_occur', 'col2558', 'col1499', 'col1475', 'col1848', 'col1845', 'col1457', 'col2307', 'col2656', 'col1500', 'col1839', 'col1850']

In [39]:
params_cat = {
    'loss_function' :'Logloss', 
    'max_depth' : 6, 
    'eval_metric' :'AUC', 
    'learning_rate' : 0.1, 
#     'grow_policy' : 'Depthwise',
    'grow_policy' : 'SymmetricTree',
    'l2_leaf_reg' : 30, 
    'random_strength' : 20,
    'random_state' : 148 ,
}

In [40]:
bst_list_cat = []
for seed in [0, 10, 50, 100, 1000]:
    params_cat['random_state'] = seed
    bst_list, pred_train, pred_test, score, fi = cat_train(train[best_cols], train['target'], len(train), split_list, params_cat, 10)
    bst_list_cat += [bst_list]

120 0.9331854700428844
1 0.9719887955182073
2 0.9586295328942388
3 0.9468855005619711
4 0.9465909415787507
5 0.9288231315504991
6 0.9290976537969305
118 0.936558652117316
1 0.9607843137254902
2 0.9567435082140965
3 0.9524657576128165
4 0.9501809876920266
5 0.9343972220018258
6 0.9362071764904248
120 0.9422511982196075
1 0.9761904761904763
2 0.9597168597168597
3 0.9545548295548295
4 0.9540899817883
5 0.938857430368793
6 0.939695079067293
105 0.9360300576544244
1 0.9712885154061625
2 0.956928041486865
3 0.9494904891963717
4 0.9496485841450133
5 0.9376701102606486
6 0.938570976391523
105 0.9318971110950717
1 0.9712885154061626
2 0.9609820955409192
3 0.9490949013007838
4 0.9485167680505379
5 0.9359360276264516
6 0.9356782130672919


In [41]:
# for bst_list in bst_list_cat:
#     for (_, val), bst in zip(split_list, bst_list[3:]):
#         pred = bst.predict(train.loc[val, best_cols])
#         np.mean(train.loc[val[np.argsort(-pred)], 'target'])

In [43]:
cat_predict = []
tmp_df = train[train.report_date >= '2023-03-01'][best_cols]
for bst_list in bst_list_cat:
    for bst in bst_list[3:]:
        cat_predict += [ bst.predict(tmp_df, prediction_type = 'Probability')[:, 1] ]
cat_predict = np.mean(cat_predict, 0)

In [44]:
submission = pd.read_csv('submission.csv', sep = ';')

In [45]:
submission['target'] = cat_predict + lgb_predict

In [46]:
submission.to_csv('submission_1.csv', index = None, sep = ';')